# Lightweight Fine-Tuning Project

In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "f35354cd",
   "metadata": {},
   "source": [
    "# Lightweight Fine-Tuning Project"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "560fb3ff",
   "metadata": {},
   "source": [
    "In this cell, describe your choices for each of the following\n",
    "\n",
    "* PEFT technique: LoRA\n",
    "* Model: gpt2\n",
    "* Evaluation approach:Transformer trainer \n",
    "* Fine-tuning dataset:sms_spam "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "de8d76bb",
   "metadata": {},
   "source": [
    "## Loading and Evaluating a Foundation Model\n",
    "\n",
    "In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "f551c63a",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install the required version of datasets in case you have an older version\n",
    "# You will need to choose \"Kernel > Restart Kernel\" from the menu after executing this cell\n",
    "#!pip install -q \"datasets==2.15.0\"\n",
    "#!pip install transformers\n",
    "#!pip install peft\n",
    "#!pip install datasets\n",
    "#!pip install pandas\n",
    "#!pip install numpy\n",
    "#!pip install scikit-learn\n",
    "#!pip install tqdm"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "f28c4a78",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the sms_spam dataset\n",
    "# See: https://huggingface.co/datasets/sms_spam\n",
    "\n",
    "from datasets import load_dataset\n",
    "\n",
    "# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test\n",
    "dataset = load_dataset(\"sms_spam\", split=\"train\").train_test_split(\n",
    "    test_size=0.2, shuffle=True, seed=23\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "e93c8ccb",
   "metadata": {},
   "outputs": [],
   "source": [
    "from transformers import AutoTokenizer\n",
    "\n",
    "tokenizer = AutoTokenizer.from_pretrained(\"gpt2\")\n",
    "tokenizer.pad_token = tokenizer.eos_token\n",
    "\n",
    "# Tokenize dataset\n",
    "def tokenize(batch):\n",
    "    return tokenizer(batch[\"sms\"], padding=True, truncation=True)\n",
    "\n",
    "# Tokenize train and test sets\n",
    "train_dataset = dataset[\"train\"].map(tokenize, batched=True)\n",
    "test_dataset = dataset[\"test\"].map(tokenize, batched=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "25689a32",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Dataset({\n",
       "    features: ['sms', 'label', 'input_ids', 'attention_mask'],\n",
       "    num_rows: 4459\n",
       "})"
      ]
     },
     "execution_count": 4,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "train_dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "568e784f",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Dataset({\n",
       "    features: ['sms', 'label', 'input_ids', 'attention_mask'],\n",
       "    num_rows: 1115\n",
       "})"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "test_dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "1e9df5e8",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']\n",
      "You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.\n"
     ]
    }
   ],
   "source": [
    "from transformers import AutoModelForSequenceClassification\n",
    "\n",
    "foundation_model = AutoModelForSequenceClassification.from_pretrained(\"gpt2\", num_labels=2,\n",
    "    id2label={0: \"not spam\", 1: \"spam\"},\n",
    "    label2id={\"not spam\": 0, \"spam\": 1},\n",
    ")\n",
    "foundation_model.config.pad_token_id = tokenizer.pad_token_id\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "aa794bbe",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "GPT2ForSequenceClassification(\n",
      "  (transformer): GPT2Model(\n",
      "    (wte): Embedding(50257, 768)\n",
      "    (wpe): Embedding(1024, 768)\n",
      "    (drop): Dropout(p=0.1, inplace=False)\n",
      "    (h): ModuleList(\n",
      "      (0-11): 12 x GPT2Block(\n",
      "        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)\n",
      "        (attn): GPT2Attention(\n",
      "          (c_attn): Conv1D()\n",
      "          (c_proj): Conv1D()\n",
      "          (attn_dropout): Dropout(p=0.1, inplace=False)\n",
      "          (resid_dropout): Dropout(p=0.1, inplace=False)\n",
      "        )\n",
      "        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)\n",
      "        (mlp): GPT2MLP(\n",
      "          (c_fc): Conv1D()\n",
      "          (c_proj): Conv1D()\n",
      "          (act): NewGELUActivation()\n",
      "          (dropout): Dropout(p=0.1, inplace=False)\n",
      "        )\n",
      "      )\n",
      "    )\n",
      "    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)\n",
      "  )\n",
      "  (score): Linear(in_features=768, out_features=2, bias=False)\n",
      ")\n"
     ]
    }
   ],
   "source": [
    "print(foundation_model)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ca02945d",
   "metadata": {},
   "source": [
    "## Evaluating gpt2 model without changing parameters"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "c3712d97",
   "metadata": {},
   "outputs": [],
   "source": [
    "from transformers import Trainer, TrainingArguments\n",
    "from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\n",
    "import torch\n",
    "\n",
    "predictions = []\n",
    "labels = []\n",
    "for example in test_dataset:    \n",
    "    device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")    \n",
    "    foundation_model.to(device)\n",
    "\n",
    "    # Prepare the input text\n",
    "    inputs = tokenizer(example[\"sms\"], return_tensors=\"pt\").to(device)\n",
    "\n",
    "    # Get predictions\n",
    "    with torch.no_grad():\n",
    "        outputs = foundation_model(**inputs)\n",
    "        logits = outputs.logits        \n",
    "\n",
    "    probabilities = torch.nn.functional.softmax(logits, dim=1)    \n",
    "    predicted_class_id = probabilities.argmax().item()\n",
    "    \n",
    "    # Here are the lists for the predicted output and the ground truth\n",
    "    predictions.append(predicted_class_id)\n",
    "    labels.append(example[\"label\"])\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "9ec48bdf",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "{'accuracy': 0.6089686098654709}\n"
     ]
    }
   ],
   "source": [
    "\n",
    "# Define function to compute metrics\n",
    "def compute_metrics(labels, preds):\n",
    "    acc = accuracy_score(labels, preds)\n",
    "    #precision = precision_score(labels, preds)\n",
    "    #recall = recall_score(labels, preds)\n",
    "    #f1 = f1_score(labels, preds)\n",
    "    return {\"accuracy\": acc}\n",
    "\n",
    "# Compute evaluation metrics\n",
    "evaluation_metrics = compute_metrics(labels, predictions)\n",
    "print(evaluation_metrics)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "4d52a229",
   "metadata": {},
   "source": [
    "## Performing Parameter-Efficient Fine-Tuning\n",
    "\n",
    "In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "id": "5775fadf",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']\n",
      "You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.\n",
      "/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.\n",
      "  warnings.warn(\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "trainable params: 150,528 || all params: 124,590,336 || trainable%: 0.1208183594592762\n"
     ]
    }
   ],
   "source": [
    "from peft import LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification\n",
    "\n",
    "# PEFT model configuration\n",
    "peft_config = LoraConfig(\n",
    "    task_type=TaskType.SEQ_CLS,\n",
    "    inference_mode=False,\n",
    "    r=4,\n",
    "    lora_alpha=16,\n",
    "    lora_dropout=0.1\n",
    ")\n",
    "\n",
    "# Load the pre-trained GPT-2 model\n",
    "\n",
    "model = AutoModelForSequenceClassification.from_pretrained(\"gpt2\", num_labels=2,\n",
    "    id2label={0: \"not spam\", 1: \"spam\"},\n",
    "    label2id={\"not spam\": 0, \"spam\": 1},\n",
    ")\n",
    "model.config.pad_token_id = model.config.eos_token_id\n",
    "\n",
    "peft_model = PeftModelForSequenceClassification(model, peft_config)\n",
    "\n",
    "# Print\n",
    "peft_model.print_trainable_parameters()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "3ec7fb35",
   "metadata": {},
   "source": [
    "## PEFT Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "2627baef",
   "metadata": {},
   "outputs": [],
   "source": [
    "def compute_metrics(eval_pred):\n",
    "    predictions, labels = eval_pred\n",
    "    predictions = np.argmax(predictions, axis=1)\n",
    "    return {\"accuracy\": (predictions == labels).mean()}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "id": "4a65581a",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "\n",
       "    <div>\n",
       "      \n",
       "      <progress value='140' max='140' style='width:300px; height:20px; vertical-align: middle;'></progress>\n",
       "      [140/140 02:56, Epoch 1/1]\n",
       "    </div>\n",
       "    <table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       " <tr style=\"text-align: left;\">\n",
       "      <th>Epoch</th>\n",
       "      <th>Training Loss</th>\n",
       "      <th>Validation Loss</th>\n",
       "      <th>Accuracy</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <td>1</td>\n",
       "      <td>0.702600</td>\n",
       "      <td>0.527249</td>\n",
       "      <td>0.869058</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table><p>"
      ],
      "text/plain": [
       "<IPython.core.display.HTML object>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "TrainOutput(global_step=140, training_loss=0.6699078832353864, metrics={'train_runtime': 177.9304, 'train_samples_per_second': 25.06, 'train_steps_per_second': 0.787, 'total_flos': 588140786128896.0, 'train_loss': 0.6699078832353864, 'epoch': 1.0})"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.\n",
    "# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer\n",
    "from transformers import DataCollatorWithPadding, Trainer, TrainingArguments\n",
    "import numpy as np\n",
    "\n",
    "peft_training_args = TrainingArguments(\n",
    "    output_dir=\"./results/peft_model\",\n",
    "    evaluation_strategy=\"epoch\",\n",
    "    learning_rate=2e-5,\n",
    "    per_device_train_batch_size=32,\n",
    "    per_device_eval_batch_size=32,\n",
    "    num_train_epochs=1,\n",
    "    weight_decay=0.01,\n",
    "    logging_dir='./logs/peft_model',\n",
    "    save_strategy=\"epoch\",\n",
    "    load_best_model_at_end=True,\n",
    "    logging_steps=100,\n",
    "    warmup_ratio=0.1,\n",
    ")\n",
    "\n",
    "# Initialize the Trainer with compute_metrics\n",
    "peft_trainer = Trainer(\n",
    "    model=peft_model,\n",
    "    args=peft_training_args,\n",
    "    train_dataset=train_dataset,\n",
    "    eval_dataset=test_dataset,\n",
    "    compute_metrics=compute_metrics,\n",
    "    tokenizer=tokenizer,\n",
    "    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),\n",
    ")\n",
    "\n",
    "peft_trainer.train()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "id": "d2efa847",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "    <div>\n",
       "      \n",
       "      <progress value='35' max='35' style='width:300px; height:20px; vertical-align: middle;'></progress>\n",
       "      [35/35 00:14]\n",
       "    </div>\n",
       "    "
      ],
      "text/plain": [
       "<IPython.core.display.HTML object>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Evaluation Results: {'eval_loss': 0.5272494554519653, 'eval_accuracy': 0.8690582959641255, 'eval_runtime': 15.1245, 'eval_samples_per_second': 73.722, 'eval_steps_per_second': 2.314, 'epoch': 1.0}\n"
     ]
    }
   ],
   "source": [
    "# Evaluate\n",
    "evaluation_results_peft = peft_trainer.evaluate()\n",
    "print(\"Evaluation Results:\", evaluation_results_peft)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "791d9dd2",
   "metadata": {},
   "source": [
    "## Save PEFT model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "f432f53f",
   "metadata": {},
   "outputs": [],
   "source": [
    "peft_model.save_pretrained('model/peft_model')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "615b12c6",
   "metadata": {},
   "source": [
    "## Performing Inference with a PEFT Model\n",
    "\n",
    "In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "id": "863ec66e",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']\n",
      "You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.\n"
     ]
    }
   ],
   "source": [
    "from peft import LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification\n",
    "\n",
    "inference_model = AutoPeftModelForSequenceClassification.from_pretrained(\n",
    "    \"model/peft_model\",\n",
    "    num_labels=2,\n",
    "    id2label={0: \"not spam\", 1: \"spam\"},\n",
    "    label2id={\"not spam\": 0, \"spam\": 1},\n",
    ")\n",
    "inference_model.config.pad_token_id = inference_model.config.eos_token_id"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "id": "bc3a8147",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "    <div>\n",
       "      \n",
       "      <progress value='18' max='18' style='width:300px; height:20px; vertical-align: middle;'></progress>\n",
       "      [18/18 00:13]\n",
       "    </div>\n",
       "    "
      ],
      "text/plain": [
       "<IPython.core.display.HTML object>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Evaluation Results: {'eval_loss': 0.5272493958473206, 'eval_accuracy': 0.8690582959641255, 'eval_runtime': 14.7069, 'eval_samples_per_second': 75.815, 'eval_steps_per_second': 1.224}\n"
     ]
    }
   ],
   "source": [
    "peft_training_args = TrainingArguments(\n",
    "    output_dir=\"./results/inference_model\",\n",
    "    evaluation_strategy=\"epoch\",\n",
    "    learning_rate=2e-5,\n",
    "    per_device_train_batch_size=32,\n",
    "    per_device_eval_batch_size=64,\n",
    "    num_train_epochs=1,\n",
    "    weight_decay=0.01,\n",
    "    logging_dir='./logs/inference_model',\n",
    "    save_strategy=\"epoch\",\n",
    "    load_best_model_at_end=True,\n",
    "    logging_steps=100,\n",
    "    warmup_ratio=0.1,\n",
    ")\n",
    "\n",
    "trainer = Trainer(\n",
    "    model=inference_model,\n",
    "    args=peft_training_args,\n",
    "    eval_dataset=test_dataset,\n",
    "    compute_metrics=compute_metrics,\n",
    "    tokenizer=tokenizer,\n",
    "    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),\n",
    ")\n",
    "\n",
    "# Evaluate the model\n",
    "evaluation_results = trainer.evaluate()\n",
    "print(\"Evaluation Results:\", evaluation_results)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "id": "bc96905a",
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "\n",
    "def predict(prompt: str) -> str:\n",
    "    device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")    \n",
    "    inference_model.to(device)\n",
    "\n",
    "    # Prepare the input text\n",
    "    inputs = tokenizer(prompt, return_tensors=\"pt\").to(device)\n",
    "\n",
    "    # Get predictions\n",
    "    with torch.no_grad():\n",
    "        outputs = inference_model(**inputs)\n",
    "        logits = outputs.logits        \n",
    "\n",
    "    probabilities = torch.nn.functional.softmax(logits, dim=1)    \n",
    "    predicted_class_id = probabilities.argmax().item()    \n",
    "    id2label={0: \"spam\", 1: \"not spam\"}\n",
    "    predicted_label = id2label[predicted_class_id]\n",
    "\n",
    "    return predicted_label"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "id": "866ab28c",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Prompt: 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE.'\n",
      "Predicted label: spam\n"
     ]
    }
   ],
   "source": [
    "# Example usage\n",
    "prompt = \"Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE.\"\n",
    "predicted_label = predict(prompt)\n",
    "print(f\"Prompt: '{prompt}'\\nPredicted label: {predicted_label}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "id": "f9a32e4e",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Prompt: 'I am Arun and want to say thanks'\n",
      "Predicted label: spam\n"
     ]
    }
   ],
   "source": [
    "# Example usage\n",
    "prompt = \"I am Arun and want to say thanks\"\n",
    "predicted_label = predict(prompt)\n",
    "print(f\"Prompt: '{prompt}'\\nPredicted label: {predicted_label}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "834deda0",
   "metadata": {},
   "source": [
    "# Conclusion:\n",
    "## Compare PEFT performance to performance of the original foundational model "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "bd9928fe",
   "metadata": {},
   "source": [
    "* A classic fine-tuning approach of Large Language Models typically changes most weights of the models which requires a lot of resources. \n",
    "* LoRA based fine-tuning is efficient way of fine-tuning as it freezes the original weights and only trains a small number of parameters making the training much more efficient.\n",
    "* When compared the performance of the original foundational model to PEFT model, then PEFT model has higher accuracy."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: gpt2
* Evaluation approach:Transformer trainer 
* Fine-tuning dataset:sms_spam 

## Loading and Evaluating a Foundation Model

In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell
#!pip install -q "datasets==2.15.0"
#!pip install transformers
#!pip install peft
#!pip install datasets
#!pip install pandas
#!pip install numpy
#!pip install scikit-learn
#!pip install tqdm

In [2]:
# Load the sms_spam dataset
# See: https://huggingface.co/datasets/sms_spam

from datasets import load_dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Tokenize dataset
def tokenize(batch):
    return tokenizer(batch["sms"], padding=True, truncation=True)

# Tokenize train and test sets
train_dataset = dataset["train"].map(tokenize, batched=True)
test_dataset = dataset["test"].map(tokenize, batched=True)

In [4]:
train_dataset

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [5]:
test_dataset

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1115
})

In [6]:
from transformers import AutoModelForSequenceClassification

foundation_model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)
foundation_model.config.pad_token_id = tokenizer.pad_token_id


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print(foundation_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


## Evaluating gpt2 model without changing parameters

In [8]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

predictions = []
labels = []
for example in test_dataset:    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
    foundation_model.to(device)

    # Prepare the input text
    inputs = tokenizer(example["sms"], return_tensors="pt").to(device)

    # Get predictions
    with torch.no_grad():
        outputs = foundation_model(**inputs)
        logits = outputs.logits        

    probabilities = torch.nn.functional.softmax(logits, dim=1)    
    predicted_class_id = probabilities.argmax().item()
    
    # Here are the lists for the predicted output and the ground truth
    predictions.append(predicted_class_id)
    labels.append(example["label"])


In [9]:

# Define function to compute metrics
def compute_metrics(labels, preds):
    acc = accuracy_score(labels, preds)
    #precision = precision_score(labels, preds)
    #recall = recall_score(labels, preds)
    #f1 = f1_score(labels, preds)
    return {"accuracy": acc}

# Compute evaluation metrics
evaluation_metrics = compute_metrics(labels, predictions)
print(evaluation_metrics)

{'accuracy': 0.6089686098654709}


## Performing Parameter-Efficient Fine-Tuning

In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [10]:
from peft import LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification

# PEFT model configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.1
)

# Load the pre-trained GPT-2 model

model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)
model.config.pad_token_id = model.config.eos_token_id

peft_model = PeftModelForSequenceClassification(model, peft_config)

# Print
peft_model.print_trainable_parameters()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 150,528 || all params: 124,590,336 || trainable%: 0.1208183594592762


## PEFT Evaluation

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [12]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np

peft_training_args = TrainingArguments(
    output_dir="./results/peft_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs/peft_model',
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    warmup_ratio=0.1,
)

# Initialize the Trainer with compute_metrics
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

peft_trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.702600,0.527249,0.869058


TrainOutput(global_step=140, training_loss=0.6699078832353864, metrics={'train_runtime': 177.9304, 'train_samples_per_second': 25.06, 'train_steps_per_second': 0.787, 'total_flos': 588140786128896.0, 'train_loss': 0.6699078832353864, 'epoch': 1.0})

In [13]:
# Evaluate
evaluation_results_peft = peft_trainer.evaluate()
print("Evaluation Results:", evaluation_results_peft)

Evaluation Results: {'eval_loss': 0.5272494554519653, 'eval_accuracy': 0.8690582959641255, 'eval_runtime': 15.1245, 'eval_samples_per_second': 73.722, 'eval_steps_per_second': 2.314, 'epoch': 1.0}


## Save PEFT model

In [14]:
peft_model.save_pretrained('model/peft_model')

## Performing Inference with a PEFT Model

In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [15]:
from peft import LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification

inference_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "model/peft_model",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)
inference_model.config.pad_token_id = inference_model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
peft_training_args = TrainingArguments(
    output_dir="./results/inference_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs/inference_model',
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    warmup_ratio=0.1,
)

trainer = Trainer(
    model=inference_model,
    args=peft_training_args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

# Evaluate the model
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

Evaluation Results: {'eval_loss': 0.5272493958473206, 'eval_accuracy': 0.8690582959641255, 'eval_runtime': 14.7069, 'eval_samples_per_second': 75.815, 'eval_steps_per_second': 1.224}


In [18]:
import torch

def predict(prompt: str) -> str:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
    inference_model.to(device)

    # Prepare the input text
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Get predictions
    with torch.no_grad():
        outputs = inference_model(**inputs)
        logits = outputs.logits        

    probabilities = torch.nn.functional.softmax(logits, dim=1)    
    predicted_class_id = probabilities.argmax().item()    
    id2label={0: "spam", 1: "not spam"}
    predicted_label = id2label[predicted_class_id]

    return predicted_label

In [19]:
# Example usage
prompt = "Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE."
predicted_label = predict(prompt)
print(f"Prompt: '{prompt}'\nPredicted label: {predicted_label}")

Prompt: 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE.'
Predicted label: spam


In [20]:
# Example usage
prompt = "I am Arun and want to say thanks"
predicted_label = predict(prompt)
print(f"Prompt: '{prompt}'\nPredicted label: {predicted_label}")

Prompt: 'I am Arun and want to say thanks'
Predicted label: spam


# Conclusion:
## Compare PEFT performance to performance of the original foundational model 

* A classic fine-tuning approach of Large Language Models typically changes most weights of the models which requires a lot of resources. 
* LoRA based fine-tuning is efficient way of fine-tuning as it freezes the original weights and only trains a small number of parameters making the training much more efficient.
* When compared the performance of the original foundational model to PEFT model, then PEFT model has higher accuracy.